# PyMentat を用いた有限要素モデルの作成

In [85]:
# PyMentatモジュールのインポート
# from py_mentat import *
import math

In [86]:
# パラメータの設定
z = [0, 5000, 10000, 15000] # 基準点(mm)
t = [270, 260, 250, 240] # 板厚(mm)
r = [1900, 1850, 1800, 1750] # 半径(mm)
n = [3, 12, 10] # (r, theta, z)方向の区間ごと分割数

In [87]:
# ループのため利用するデータを区間ごとにまとめておく
z1 = z[0:len(z)-1]
z2 = z[1:len(z)]
t1 = t[0:len(t)-1]
t2 = t[1:len(t)]
r1 = r[0:len(r)-1]
r2 = r[1:len(r)]

data = []
for i in range(0, len(z1)):
    data.append([z1[i], z2[i], t1[i], t2[i], r1[i], r2[i]])

In [88]:
# 座標値の作成

nodes = []
# 区間ごとにループを実行(最後の区間は別に)
for (z1, z2, t1, t2, r1, r2) in data[:len(data)-1]:
    # z座標を分割
    z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2])]
    
    # z座標におけるr, tを計算
    t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2])]
    r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2])]
    
    for k in range(len(z_list)):
        # z
        zz = z_list[k]
        
        # r方向の分割値
        rr = r_list[k]
        tt = t_list[k]
        dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
        # theta方向の分割値(radian)
        rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
        for dr in dr_list:
            for rad in rad_list:
                x = dr * math.sin(rad)
                y = dr * math.cos(rad)
                nodes.append([x, y, zz])
        
# 最後の区間
(z1, z2, t1, t2, r1, r2) = data[-1]

# z座標を分割
z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2]+1)]
    
# z座標におけるr, tを計算
t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2]+1)]
r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2]+1)]
    
for k in range(len(z_list)):
    # z
    zz = z_list[k]
        
    # r方向の分割値
    rr = r_list[k]
    tt = t_list[k]
    dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
    # theta方向の分割値(radian)
    rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
    for dr in dr_list:
        for rad in rad_list:
            x = dr * math.sin(rad)
            y = dr * math.cos(rad)
            nodes.append([x, y, zz])

1488


In [94]:
# 要素の作成
n_blocks = len(z) - 1

num_z = n[2]
num_r = n[0]
num_theta = n[1]
num_layer = (num_r + 1) * num_theta # 各層ごとの節点数

elems = []
for k in range(num_z * n_blocks): # z方向
    for j in range(num_r): # r方向
        for i in range(num_theta): # theta方向
            if i == num_theta - 1: # 1周回ったら補正
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+1, o-num_theta+1, o+num_layer, o+num_layer+num_theta, o+num_layer+1, o+num_layer-num_theta+1]
            else:
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+num_theta+1, o+1, o+num_layer, o+num_layer+num_theta, o+num_layer+num_theta+1, o+num_layer+1]
            elems.append(e)

0 0 0 [1, 13, 14, 2, 49, 61, 62, 50]
1 0 0 [2, 14, 15, 3, 50, 62, 63, 51]
2 0 0 [3, 15, 16, 4, 51, 63, 64, 52]
3 0 0 [4, 16, 17, 5, 52, 64, 65, 53]
4 0 0 [5, 17, 18, 6, 53, 65, 66, 54]
5 0 0 [6, 18, 19, 7, 54, 66, 67, 55]
6 0 0 [7, 19, 20, 8, 55, 67, 68, 56]
7 0 0 [8, 20, 21, 9, 56, 68, 69, 57]
8 0 0 [9, 21, 22, 10, 57, 69, 70, 58]
9 0 0 [10, 22, 23, 11, 58, 70, 71, 59]
10 0 0 [11, 23, 24, 12, 59, 71, 72, 60]
11 0 0 [12, 24, 13, 1, 60, 72, 61, 49]
0 1 0 [13, 25, 26, 14, 61, 73, 74, 62]
1 1 0 [14, 26, 27, 15, 62, 74, 75, 63]
2 1 0 [15, 27, 28, 16, 63, 75, 76, 64]
3 1 0 [16, 28, 29, 17, 64, 76, 77, 65]
4 1 0 [17, 29, 30, 18, 65, 77, 78, 66]
5 1 0 [18, 30, 31, 19, 66, 78, 79, 67]
6 1 0 [19, 31, 32, 20, 67, 79, 80, 68]
7 1 0 [20, 32, 33, 21, 68, 80, 81, 69]
8 1 0 [21, 33, 34, 22, 69, 81, 82, 70]
9 1 0 [22, 34, 35, 23, 70, 82, 83, 71]
10 1 0 [23, 35, 36, 24, 71, 83, 84, 72]
11 1 0 [24, 36, 25, 13, 72, 84, 73, 61]
0 2 0 [25, 37, 38, 26, 73, 85, 86, 74]
1 2 0 [26, 38, 39, 27, 74, 86, 87, 75]


9 1 24 [1174, 1186, 1187, 1175, 1222, 1234, 1235, 1223]
10 1 24 [1175, 1187, 1188, 1176, 1223, 1235, 1236, 1224]
11 1 24 [1176, 1188, 1177, 1165, 1224, 1236, 1225, 1213]
0 2 24 [1177, 1189, 1190, 1178, 1225, 1237, 1238, 1226]
1 2 24 [1178, 1190, 1191, 1179, 1226, 1238, 1239, 1227]
2 2 24 [1179, 1191, 1192, 1180, 1227, 1239, 1240, 1228]
3 2 24 [1180, 1192, 1193, 1181, 1228, 1240, 1241, 1229]
4 2 24 [1181, 1193, 1194, 1182, 1229, 1241, 1242, 1230]
5 2 24 [1182, 1194, 1195, 1183, 1230, 1242, 1243, 1231]
6 2 24 [1183, 1195, 1196, 1184, 1231, 1243, 1244, 1232]
7 2 24 [1184, 1196, 1197, 1185, 1232, 1244, 1245, 1233]
8 2 24 [1185, 1197, 1198, 1186, 1233, 1245, 1246, 1234]
9 2 24 [1186, 1198, 1199, 1187, 1234, 1246, 1247, 1235]
10 2 24 [1187, 1199, 1200, 1188, 1235, 1247, 1248, 1236]
11 2 24 [1188, 1200, 1189, 1177, 1236, 1248, 1237, 1225]
0 0 25 [1201, 1213, 1214, 1202, 1249, 1261, 1262, 1250]
1 0 25 [1202, 1214, 1215, 1203, 1250, 1262, 1263, 1251]
2 0 25 [1203, 1215, 1216, 1204, 1251, 1263, 